# Lab 8 - Combining Attendance and Practice Quiz Attempts

## Background

In all of my class, I use an attendace quiz to track student attendance.  Note that students take multiple attempts at the same quiz, one per class; so that number of attempts a student takes on this quiz represents the number of class session that student has attended.

In some, but not all, of my courses I also provide practice quizzes that students can use to prepare for actual quizzes and tests.  These quizzes pull questions randomly from a bank of questions, allow students unlimited attempts, and are not used as part of the students grade.

In this lab, you will collect simulated data from mock classes into one table and in the next lab you will summarize these data.

## Tasks 

The files found in `attendance_example.zip` contains (made-up and random) examples of the D2L files that I use to summarize my attendance quizzes and practice quizzes
Make sure you download `attendance_example.zip` to the `data` folder inside the course repository, then unzip the file.

1. Use `glob` to find the path to all csv files.
2. Use write functions that use regular expressions to extract the class name, quiz type (`Attendance` and `Practice`), and the module number (if the file is a practice quiz.
3. Write a function that takes a path as an argument and returns a dataframe that contains:
    * All of the original columns
    * A Class column that holds the class identifier
    * A Category column that contains the quiz type
    * A Module column that (a) contains the module number for a practice quiz, or (b) is otherwise empty.
4. Use a loop, `union`, and the accumulator pattern to load all of the data into a single table.
5. Write the resulting table to a csv file.

In [144]:
import pandas as pd
from dfply import *

In [145]:
from glob import glob
files = glob('./data/attendance_example/*/*.csv')
files

['./data/attendance_example/stat491s1/Attendance Quiz - User Attempts.csv',
 './data/attendance_example/stat491s1/Practice Quiz - Module 4 - User Attempts.csv',
 './data/attendance_example/stat491s1/Practice Quiz - Module 2 - User Attempts.csv',
 './data/attendance_example/stat491s1/Practice Quiz - Module 3 - User Attempts.csv',
 './data/attendance_example/stat491s1/Practice Quiz - Module 1 - User Attempts.csv',
 './data/attendance_example/dsci494s7/Attendance Quiz - User Attempts.csv',
 './data/attendance_example/dsci494s7/Practice Quiz - Module 4 - User Attempts.csv',
 './data/attendance_example/dsci494s7/Practice Quiz - Module 2 - User Attempts.csv',
 './data/attendance_example/dsci494s7/Practice Quiz - Module 3 - User Attempts.csv',
 './data/attendance_example/dsci494s7/Practice Quiz - Module 1 - User Attempts.csv',
 './data/attendance_example/stat180s18/Attendance Quiz - User Attempts.csv']

In [146]:
import re
CLASS_NAME_RE = re.compile(r'^\./data/attendance_example/([a-z]*[0-9]*).*\.csv$')
class_name = lambda p: CLASS_NAME_RE.match(p).group(1)
class_names = lambda files: [class_name(p) for p in files]
class_names(files)

['stat491',
 'stat491',
 'stat491',
 'stat491',
 'stat491',
 'dsci494',
 'dsci494',
 'dsci494',
 'dsci494',
 'dsci494',
 'stat180']

In [147]:
CLASS_TYPE_RE = re.compile(r'^\./data/attendance_example/.*/(\w+).*\.csv$')
class_type = lambda p: CLASS_TYPE_RE.match(p).group(1)
class_types = lambda files: [class_type(p) for p in files]
class_types(files)

['Attendance',
 'Practice',
 'Practice',
 'Practice',
 'Practice',
 'Attendance',
 'Practice',
 'Practice',
 'Practice',
 'Practice',
 'Attendance']

In [148]:
MODULE_NUM_RE = re.compile(r'^\./data/attendance_example/.*/.*(\d+).*\.csv$')
module_num = lambda p: MODULE_NUM_RE.match(p).group(1) if class_type(p)=='Practice' else "" 
module_nums = lambda files: [module_num(p) if class_type(p)=='Practice' else "" for p in files]
module_nums(files)

['', '4', '2', '3', '1', '', '4', '2', '3', '1', '']

In [149]:
att_pr_df_gen = lambda path: (pd.read_csv(path)
                              >> mutate(Class = class_name(path),
                                        Type = class_type(path),
                                        Module = module_num(path)))


In [150]:
@dfpipe
def union_all(left_df, right_df, ignore_index=True):
    return pd.concat([left_df, right_df], ignore_index=ignore_index)

In [151]:
columns = ['Org Defined ID', 'UserName', 'FirstName', 'LastName', 'Attempt #',
       'Score', 'Out Of', 'Attempt_Start', 'Attempt_End', 'Percent', 'Class',
       'Type', 'Module']
att_pr_df = pd.DataFrame(columns=columns)

In [152]:
for file in files:
    att_pr_df = att_pr_df >> union_all(att_pr_df_gen(file))
att_pr_df.shape

(3359, 13)

In [153]:
att_pr_df.to_csv("./data/attendace_pratice.csv", header = True, index = None)